In [1]:
import numpy as np
import csv
import pandas as pd

In [2]:
#Liste contenant les noms des colonnes de la Dataframe
col_names = ['case_month','res_state', 'state_fips_code''res_county' ,'county_fips_code' ,'age_group' ,'sex' ,'race' ,'ethnicity','case_positive_specimen_interval' ,'case_onset_interval','process' ,'exposure_yn' ,'current_status','symptom_status','hosp_yn', 'icu_yn','death_yn' ,'underlying_conditions_yn' ]

#Import des données
df = pd.read_csv('CDC_Data_orig.csv', encoding='ISO-8859-1', header=0, names=col_names, index_col=None, low_memory=False)

In [3]:
df_encoder = df.copy()

In [4]:
#On encode les données pour que les modéles puis les traiter
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for colonne in df.columns :
    df_encoder[colonne] = le.fit_transform(df[colonne])
    
df_encoder['death_yn'].head(100)


2020-04    0
2020-07    0
2020-11    0
2020-08    0
2020-07    0
          ..
2020-11    0
2020-06    0
2020-10    0
2020-06    0
2020-07    0
Name: death_yn, Length: 100, dtype: int32

In [5]:
df.head(100)

,case_month,res_state,state_fips_coderes_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
2020-04,KS,20,FORD,20057,18 to 49 years,Male,Unknown,Hispanic/Latino,0,0,Unknown,Unknown,Laboratory-confirmed case,Symptomatic,No,No,No,Yes
2020-07,KS,20,WYANDOTTE,20209,18 to 49 years,Female,Unknown,Hispanic/Latino,0,0,Contact tracing of case patient,Yes,Laboratory-confirmed case,Symptomatic,No,No,No,No
2020-11,KS,20,HARVEY,20079,18 to 49 years,Male,White,Hispanic/Latino,1,0,Multiple,Yes,Probable Case,Symptomatic,No,No,No,No
2020-08,KS,20,LYON,20111,18 to 49 years,Female,White,Hispanic/Latino,0,0,Routine surveillance,Yes,Laboratory-confirmed case,Symptomatic,No,No,No,No
2020-07,KS,20,SALINE,20169,18 to 49 years,Male,White,Hispanic/Latino,0,0,Laboratory reported,Unknown,Laboratory-confirmed case,Symptomatic,No,Unknown,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11,KS,20,WYANDOTTE,20209,0 - 17 years,Female,White,Hispanic/Latino,0,0,Clinical evaluation,Yes,Laboratory-confirmed case,Symptomatic,No,No,No,No
2020-06,KS,20,CRAWFORD,20037,18 to 49 years,Female,White,Hispanic/Latino,0,0,Clinical evaluation,Yes,Laboratory-confirmed case,Symptomatic,No,No,No,Yes
2020-10,KS,20,SHAWNEE,20177,18 to 49 years,Male,White,Hispanic/Latino,1,0,Laboratory reported,Yes,Laboratory-confirmed case,Symptomatic,No,No,No,No
2020-06,KS,20,SEWARD,20175,50 to 64 years,Female,White,Hispanic/Latino,1,0,Clinical evaluation,Yes,Probable Case,Symptomatic,No,No,No,Yes


In [6]:
#La fonction calcule le score ROC AUC
def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        #création d'une liste de toutes les classes à l'exception de la classe actuelle 
        other_class = [x for x in unique_class if x != per_class]

        #met la classe actuelle à 1 et les autres à 0
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]

        #utilisation de la méthode sklearn metrics pour calculer le roc_auc_score
        roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc
    
    check = pd.DataFrame(roc_auc_dict.items())
    return np.mean(check)

##########################  SVM Classifier  ################################
def svm_fun(X_train,y_train,X_test,y_test):
    #Création d'un classifeur SVM
    clf = svm.SVC(kernel='linear')

    #Entrainement du modele
    clf.fit(X_train, y_train)

    #Prediction de la reponse avec le set de test
    y_pred = clf.predict(X_test)
    
    svm_acc = metrics.accuracy_score(y_test, y_pred)    
    svm_prec = metrics.precision_score(y_test, y_pred,average='weighted') 
    svm_recall = metrics.recall_score(y_test, y_pred,average='weighted')
    svm_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')    
    svm_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')   
    svm_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix SVM : \n", confuse)
    print("SVM Class Wise Accuracy : ",confuse.diagonal()/confuse.sum(axis=1))
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    check = [svm_acc,svm_prec,svm_recall,svm_f1_weighted,svm_f1_macro,svm_f1_micro,macro_roc_auc_ovo[1]]
    return(check)
    

##########################  NB Classifier  ################################
def gaus_nb_fun(X_train,y_train,X_test,y_test):
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    NB_acc = metrics.accuracy_score(y_test, y_pred)
    NB_prec = metrics.precision_score(y_test, y_pred,average='weighted')    
    NB_recall = metrics.recall_score(y_test, y_pred,average='weighted')
    NB_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')
    NB_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')
    NB_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix NB : \n", confuse)
    print("NB Class Wise Accuracy : ",confuse.diagonal()/confuse.sum(axis=1))
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    check = [NB_acc,NB_prec,NB_recall,NB_f1_weighted,NB_f1_macro,NB_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

##########################  MLP Classifier  ################################
def mlp_fun(X_train,y_train,X_test,y_test):
    scaler = StandardScaler()  
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)  
    X_test_2 = scaler.transform(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)  
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test_2)
    
    MLP_acc = metrics.accuracy_score(y_test, y_pred)    
    MLP_prec = metrics.precision_score(y_test, y_pred,average='weighted') 
    MLP_recall = metrics.recall_score(y_test, y_pred,average='weighted')    
    MLP_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')    
    MLP_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')    
    MLP_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix MLP : \n", confuse)
    print("MLP Class Wise Accuracy : ",confuse.diagonal()/confuse.sum(axis=1))
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')    
    check = [MLP_acc,MLP_prec,MLP_recall,MLP_f1_weighted,MLP_f1_macro,MLP_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

##########################  knn Classifier  ################################
def knn_fun(X_train,y_train,X_test,y_test):
    classifier = KNeighborsClassifier(n_neighbors=5)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    knn_acc = metrics.accuracy_score(y_test, y_pred)    
    knn_prec = metrics.precision_score(y_test, y_pred,average='weighted')    
    knn_recall = metrics.recall_score(y_test, y_pred,average='weighted')   
    knn_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')   
    knn_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')   
    knn_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix KNN : \n", confuse)
    print("KNN Class Wise Accuracy : ",confuse.diagonal()/confuse.sum(axis=1))
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')  
    check = [knn_acc,knn_prec,knn_recall,knn_f1_weighted,knn_f1_macro,knn_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

##########################  Random Forest Classifier  ################################
def rf_fun(X_train,y_train,X_test,y_test):
    # Import the model we are using
    from sklearn.ensemble import RandomForestClassifier
    # Instanciation du model avec 1000 arbres de decision
    rf = RandomForestClassifier(n_estimators = 100)
    # Entrainement du modele
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    fr_acc = metrics.accuracy_score(y_test, y_pred)   
    fr_prec = metrics.precision_score(y_test, y_pred,average='weighted')   
    fr_recall = metrics.recall_score(y_test, y_pred,average='weighted')   
    fr_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')  
    fr_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')    
    fr_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix RF : \n", confuse)
    print("RF Class Wise Accuracy : ",confuse.diagonal()/confuse.sum(axis=1))
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')   
    check = [fr_acc,fr_prec,fr_recall,fr_f1_weighted,fr_f1_macro,fr_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

##########################  Logistic Regression Classifier  ################################
def lr_fun(X_train,y_train,X_test,y_test):

    model = LogisticRegression(solver='liblinear', random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    LR_acc = metrics.accuracy_score(y_test, y_pred)
    LR_prec = metrics.precision_score(y_test, y_pred,average='weighted')    
    LR_recall = metrics.recall_score(y_test, y_pred,average='weighted')    
    LR_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')   
    LR_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')   
    LR_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix LR : \n", confuse)
    print("LR Class Wise Accuracy : ",confuse.diagonal()/confuse.sum(axis=1))
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')    
    check = [LR_acc,LR_prec,LR_recall,LR_f1_weighted,LR_f1_macro,LR_f1_micro,macro_roc_auc_ovo[1]]
    return(check)


def fun_decision_tree(X_train,y_train,X_test,y_test):
    from sklearn import tree
    
    clf = tree.DecisionTreeClassifier()    
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    dt_acc = metrics.accuracy_score(y_test, y_pred)    
    dt_prec = metrics.precision_score(y_test, y_pred,average='weighted')    
    dt_recall = metrics.recall_score(y_test, y_pred,average='weighted')    
    dt_f1_weighted = metrics.f1_score(y_test, y_pred,average='weighted')    
    dt_f1_macro = metrics.f1_score(y_test, y_pred,average='macro')    
    dt_f1_micro = metrics.f1_score(y_test, y_pred,average='micro')
    
    confuse = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix DT : \n", confuse)
    print("DT Class Wise Accuracy : ",confuse.diagonal()/confuse.sum(axis=1))
    y_prob = y_pred
    macro_roc_auc_ovo = roc_auc_score_multiclass(y_test, y_prob, average='macro')
    
    check = [dt_acc,dt_prec,dt_recall,dt_f1_weighted,dt_f1_macro,dt_f1_micro,macro_roc_auc_ovo[1]]
    return(check)

In [7]:
X = df_encoder.drop(['death_yn'], axis=1).to_numpy()
y = df_encoder['death_yn'].to_numpy()

In [8]:
counter = 1

svm_table = []
gauu_nb_table = []
mlp_table = []
knn_table = []
rf_table = []
lr_table = []
dt_table = []

#On sépare la base de données en train set et test set
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
sss = ShuffleSplit(n_splits=1, test_size=0.3)
sss.get_n_splits(X, y)
train_index, test_index = next(sss.split(X, y)) 

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [9]:
X_train.shape,X_test.shape

((67188, 17), (28796, 17))

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn import svm
import timeit

start = timeit.default_timer()
gauu_nb_return = gaus_nb_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("NB Time : ", stop - start) 

start = timeit.default_timer()
mlp_return = mlp_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("MLP Time : ", stop - start) 

start = timeit.default_timer()
knn_return = knn_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("KNN Time : ", stop - start) 

start = timeit.default_timer()
rf_return = rf_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("RF Time : ", stop - start) 

start = timeit.default_timer()
lr_return = lr_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("LR Time : ", stop - start) 

start = timeit.default_timer()
dt_return = fun_decision_tree(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("DT Time : ", stop - start) 

start = timeit.default_timer()
svm_return = svm_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("SVM Time : ", stop - start) 

gauu_nb_table.append(gauu_nb_return)
mlp_table.append(mlp_return)
knn_table.append(knn_return)
rf_table.append(rf_return)
lr_table.append(lr_return)
dt_table.append(dt_return)
svm_table.append(svm_return)
     
svm_table_final = pd.DataFrame(svm_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
gauu_nb_table_final = pd.DataFrame(gauu_nb_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
mlp_table_final = pd.DataFrame(mlp_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
knn_table_final = pd.DataFrame(knn_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
rf_table_final = pd.DataFrame(rf_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
lr_table_final = pd.DataFrame(lr_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])

dt_table_final = pd.DataFrame(dt_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])

Confusion Matrix NB : 
 [[18027  6040  2753]
 [  171  1353    65]
 [   15    41   331]]
NB Class Wise Accuracy :  [0.67214765 0.85147892 0.85529716]
NB Time :  0.17420999999999953


C:\Users\akims\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Confusion Matrix MLP : 
 [[26207   561    52]
 [  818   771     0]
 [  308    17    62]]
MLP Class Wise Accuracy :  [0.97714392 0.48521082 0.16020672]
MLP Time :  43.6954417


C:\Users\akims\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Pour le NB, les scores sont sensiblements pareil mis à part pour "Yes" qui est supérieur de 0.27.

Pour le MLP, les scores sont à peu prés pareil mis à part pour "Unknown" qui est inférieur à 0.38.

Pour le KNN, les scores sont pareils à 0.06 prés.

Pour le LR les scores sont pareils à 0.1 prés.

Pour le DT, les scores sont à peu prés pareil mis à part pour "Unknown" qui est inférieur à 0.16.

In [ ]:
#taking average of all k-fold performance values
final_mean_mat = []

final_mean_mat.append(np.transpose((list(svm_table_final.mean()))))
final_mean_mat.append(np.transpose((list(gauu_nb_table_final.mean()))))
final_mean_mat.append(np.transpose((list(mlp_table_final.mean()))))
final_mean_mat.append(np.transpose((list(knn_table_final.mean()))))
final_mean_mat.append(np.transpose((list(rf_table_final.mean()))))
final_mean_mat.append(np.transpose((list(lr_table_final.mean()))))
final_mean_mat.append(np.transpose((list(dt_table_final.mean()))))

final_avg_mat = pd.DataFrame(final_mean_mat,columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"], 
                          index=["SVM","NB","MLP","KNN","RF","LR","DT"])

final_avg_mat

Les scores sont du même ordre ici et dans l'étude.

## Boruta for Feature Selection

In [ ]:
X_df = pd.DataFrame(X)
X_df.columns = ["c1","c2","c3","c4","c5","c6","c7","c8","c9","c10","c11","c12","c13","c14","c15","c16","c17"]
X_df

In [ ]:
np.random.seed(42)
X_shadow = X_df.apply(np.random.permutation)
X_shadow.columns = ['shadow_' + feat for feat in X_df.columns]
### make X_boruta by appending X_shadow to X
X_boruta = pd.concat([X_df, X_shadow], axis = 1)

In [ ]:
X_boruta

In [ ]:
from sklearn.ensemble import RandomForestRegressor
### Entrain le modele random forest
forest = RandomForestRegressor(max_depth = 5, random_state = 42)
forest.fit(X_boruta, y)
### Enregistrer les importances des caractéristiques
feat_imp_X = forest.feature_importances_[:len(X_df.columns)]
feat_imp_shadow = forest.feature_importances_[len(X_df.columns):]
### calcule de hits hits
hits = feat_imp_X > feat_imp_shadow.max()

In [ ]:
hits

In [ ]:
indices_tmp = []

for i in range(len(hits)):
    if(hits[i]==True):
        indices_tmp.append(i)

In [ ]:
final_boruta_data = np.array(X_df.iloc[:,indices_tmp])

In [ ]:
X = final_boruta_data
y = df_encoder['death_yn'].to_numpy()


counter = 1
svm_table = []
gauu_nb_table = []
mlp_table = []
knn_table = []
rf_table = []
lr_table = []
dt_table = []


from sklearn.model_selection import ShuffleSplit 
sss = ShuffleSplit(n_splits=1, test_size=0.3)
sss.get_n_splits(X, y)
train_index, test_index = next(sss.split(X, y)) 

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [ ]:
import timeit

start = timeit.default_timer()
gauu_nb_return = gaus_nb_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("NB Time : ", stop - start) 

start = timeit.default_timer()
mlp_return = mlp_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("MLP Time : ", stop - start) 

start = timeit.default_timer()
knn_return = knn_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("KNN Time : ", stop - start) 

start = timeit.default_timer()
rf_return = rf_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("RF Time : ", stop - start) 

start = timeit.default_timer()
lr_return = lr_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("LR Time : ", stop - start) 

start = timeit.default_timer()
dt_return = fun_decision_tree(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("DT Time : ", stop - start) 

gauu_nb_table.append(gauu_nb_return)
mlp_table.append(mlp_return)
knn_table.append(knn_return)
rf_table.append(rf_return)
lr_table.append(lr_return)
dt_table.append(dt_return)
     
gauu_nb_table_final = pd.DataFrame(gauu_nb_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
mlp_table_final = pd.DataFrame(mlp_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
knn_table_final = pd.DataFrame(knn_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
rf_table_final = pd.DataFrame(rf_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
lr_table_final = pd.DataFrame(lr_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])

dt_table_final = pd.DataFrame(dt_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])

Avec cette selection de feature, le score de l'accuracy ne bouge pas.

In [ ]:
final_mean_mat = []

final_mean_mat.append(np.transpose((list(gauu_nb_table_final.mean()))))
final_mean_mat.append(np.transpose((list(mlp_table_final.mean()))))
final_mean_mat.append(np.transpose((list(knn_table_final.mean()))))
final_mean_mat.append(np.transpose((list(rf_table_final.mean()))))
final_mean_mat.append(np.transpose((list(lr_table_final.mean()))))
final_mean_mat.append(np.transpose((list(dt_table_final.mean()))))

final_avg_mat = pd.DataFrame(final_mean_mat,columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"], 
                          index=["NB","MLP","KNN","RF","LR","DT"])

final_avg_mat

In [ ]:
Pareil pour les autres metrics.

## Ridge Regression


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

X = df_encoder.drop(['death_yn'], axis=1).to_numpy()
y = df_encoder['death_yn'].to_numpy()

ridge_data = df_encoder.drop(['death_yn'], axis=1)
                             
X_train, X_test, y_train, y_test = train_test_split(
    ridge_data,y,
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

# L1 = Lasso, L2 = Ridge
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l2', solver='liblinear'))
sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

sel_.get_support()

selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

# L1 = Lasso, L2 = Ridge
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l2', solver='liblinear'))
sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

sel_.get_support()

selected_feat_ridge = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat_ridge)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

In [ ]:
Ridge_Regression_data = sel_.transform(ridge_data.fillna(0))


In [ ]:
X = Ridge_Regression_data


counter = 1
svm_table = []
gauu_nb_table = []
mlp_table = []
knn_table = []
rf_table = []
lr_table = []
dt_table = []


from sklearn.model_selection import ShuffleSplit 
sss = ShuffleSplit(n_splits=1, test_size=0.3)
sss.get_n_splits(X, y)
train_index, test_index = next(sss.split(X, y)) 

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [ ]:
import timeit

start = timeit.default_timer()
gauu_nb_return = gaus_nb_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("NB Time : ", stop - start) 

start = timeit.default_timer()
mlp_return = mlp_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("MLP Time : ", stop - start) 

start = timeit.default_timer()
knn_return = knn_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("KNN Time : ", stop - start) 

start = timeit.default_timer()
rf_return = rf_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("RF Time : ", stop - start) 

start = timeit.default_timer()
lr_return = lr_fun(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("LR Time : ", stop - start) 

start = timeit.default_timer()
dt_return = fun_decision_tree(X_train,y_train,X_test,y_test)
stop = timeit.default_timer()
print("DT Time : ", stop - start) 

gauu_nb_table.append(gauu_nb_return)
mlp_table.append(mlp_return)
knn_table.append(knn_return)
rf_table.append(rf_return)
lr_table.append(lr_return)
dt_table.append(dt_return)
# svm_table.append(svm_return)

gauu_nb_table_final = pd.DataFrame(gauu_nb_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
mlp_table_final = pd.DataFrame(mlp_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
knn_table_final = pd.DataFrame(knn_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
rf_table_final = pd.DataFrame(rf_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])
lr_table_final = pd.DataFrame(lr_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])

dt_table_final = pd.DataFrame(dt_table, columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"])

In [ ]:
#taking average of all k-fold performance values
final_mean_mat = []

# final_mean_mat.append(np.transpose((list(svm_table_final.mean()))))
final_mean_mat.append(np.transpose((list(gauu_nb_table_final.mean()))))
final_mean_mat.append(np.transpose((list(mlp_table_final.mean()))))
final_mean_mat.append(np.transpose((list(knn_table_final.mean()))))
final_mean_mat.append(np.transpose((list(rf_table_final.mean()))))
final_mean_mat.append(np.transpose((list(lr_table_final.mean()))))
final_mean_mat.append(np.transpose((list(dt_table_final.mean()))))

final_avg_mat = pd.DataFrame(final_mean_mat,columns=["Accuracy","Precision","Recall",
                                                "F1 (weighted)","F1 (Macro)","F1 (Micro)","ROC AUC"], 
                          index=["NB","MLP","KNN","RF","LR","DT"])

final_avg_mat

Pour le NB, accuracy, recall, F1 (weighted) meilleur de 0.1 et un moins bon ROC.

Pour le reste c'est à peu prés pareil si ce n'est "F1 (Macro)" qui est inférieur de 0.1.